In [1]:
import json
import pandas as pd

In [2]:
dfs = []

In [3]:
for project in ['cinder', 'glance', 'neutron']:
    df = pd.read_csv(f'../data/rq1/rq1_{project}.csv')
    df['project'] = project
    dfs.append(df)

In [4]:
df = pd.concat(dfs, ignore_index=True)

In [5]:
len(df)

107213

## Patch - fill in NANs

In [6]:
df.isna().sum()

hash                                       0
author_name                                0
committer_name                             0
author_date                                0
commit_date                                0
num_lines_added                            0
num_lines_deleted                          0
num_lines_of_code                          0
num_file_impacted                          0
num_dirs_impacted                          0
min_complexity                             0
mean_complexity                            0
max_complexity                             0
entropy                                    0
bug_fixing                                 0
description_length                         0
num_prior_commits                          0
avg_prior_age                              0
num_prior_commits_bug_fixing               0
num_future_commits_bug_fixing              0
fix_inducing                               0
reviewer_id                                0
reviewer_n

In [7]:
df = df.fillna(0)

## Patch - fill in `author_is_core`

In [8]:
with open('../data/core_devs.json', 'r') as j:
    CORE_DEVS = json.loads(j.read())

In [9]:
CORE_DEVS

{'cinder': ['Brian Rosmaita',
  'Eric Harney',
  'Gorka Eguileor',
  'Ivan Kolodyazhny',
  'Jay Bryant',
  'Lucio Seki',
  'Rajat Dhasmana',
  'Sean McGinnis',
  'Hemna'],
 'glance': ['Abhishek Kekane',
  'Brian Rosmaita',
  'Dan Smith',
  'Glance Bot',
  'Nikhil Komawar',
  'Sean McGinnis'],
 'neutron': ['Akihiro Motoki',
  'Brian Haley',
  'Hongbin Lu',
  'Jakub Libosvar',
  'Lajos Katona',
  'Miguel Lavalle',
  'Nate Johnston',
  'Oleg Bondarev',
  'Rodolfo Alonso',
  'Slawek Kaplonski',
  'YAMAMOTO Takashi',
  'LIU Yulong'],
 'nova': ['Alex Xu',
  'Balazs Gibizer',
  'Dan Smith',
  'Eric Fried',
  'Ghanshyam',
  'John Garbutt',
  'Lee Yarwood',
  'stephenfin',
  'Sylvain Bauza',
  'melwitt']}

In [10]:
for idx, row in df.iterrows():
    project = row['project']
    project_core_devs = CORE_DEVS[project]
    author = row['author_name']
    if author in project_core_devs:
        df.at[idx, 'author_is_core'] = True
    else:
        df.at[idx, 'author_is_core'] = False

In [11]:
df['author_is_core'].value_counts()

False    94843
True     12370
Name: author_is_core, dtype: int64

## Target variable

In [12]:
df['reviewer_vote'].value_counts()

 2    47888
 1    47795
-1    10801
-2      729
Name: reviewer_vote, dtype: int64

In [13]:
df['pos_vote'] = df['reviewer_vote'] > 0

In [14]:
df['pos_vote'].value_counts()

True     95683
False    11530
Name: pos_vote, dtype: int64

## Export

In [15]:
df.to_csv('../data/rq1/rq1_all.csv', index=False)